# NGSolve - PETSc interface
We learn how we can interface the popular parallel toolkit PETSc for solving linear equations and much more. We use the Python interface petsc4py. The whole Python file is [ngs2petsc.py](ngs2petsc.py)

In [ ]:
from ipyparallel import Client
c = Client(profile='mpi')

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square
comm = MPI.COMM_WORLD
if comm.rank == 0:
    ngmesh = unit_square.GenerateMesh(maxh=0.1).Distribute(comm)
else:
    ngmesh = netgen.meshing.Mesh.Receive(comm)
    
for l in range(0):
    ngmesh.Refine()
mesh = Mesh(ngmesh)

In [ ]:
%%px
import numpy as np
import petsc4py.PETSc as psc

In [ ]:
%%px
fes = H1(mesh, order=1)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx+u*v*ds).Assemble()
f = LinearForm(x*v*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, freedofs=fes.FreeDofs(), printing=False, maxsteps=400, tol=1e-8)
gfu.vec.data = inv * f.vec
print ("ngs-dot =", InnerProduct(gfu.vec, f.vec) )

We convert the local sparse matrix to a local PETSc AIJ matrix:

In [ ]:
%%px
locmat = a.mat.local_mat
val,col,ind = locmat.CSR()
ind = np.array(ind, dtype='int32')

apsc_loc = psc.Mat().createAIJ(size=(locmat.height, locmat.width), csr=(ind,col,val), comm=MPI.COMM_SELF)

The NGSolve ParallelDofs object corresponds to a PETSc IndexSet object, which we create next. In PETSc, dofs are globally enumerated, what is not the case in NGSolve. For this purpose, a ParallelDofs class can generate a globally consistent enumeration of dofs. The generated globnums array contains the global dof-numbers of the local dofs.

In [ ]:
%%px
pardofs = fes.ParallelDofs()
globnums, nglob = pardofs.EnumerateGlobally() 
print (list(globnums))

iset = psc.IS().createGeneral (indices=globnums, comm=comm)
lgmap = psc.LGMap().createIS(iset)

In [ ]:
%%px
help (psc.IS)

We can now create the global matrix using our local2global map:

In [ ]:
%%px
mat = psc.Mat().createPython(size=nglob, comm=comm)
mat.setType(psc.Mat.Type.IS)
mat.setLGMap(lgmap)
mat.setISLocalMat(apsc_loc)
mat.assemble()

In [ ]:
%%px

f.vec.Cumulate()

v1, v2 = mat.createVecs()

v2loc = v2.getSubVector(iset)
v2loc.getArray()[:] = f.vec.FV()
v2.restoreSubVector(IS, v2loc)

In [ ]:
%%px
ksp = psc.KSP()
ksp.create()
ksp.setOperators(mat)
ksp.setType(psc.KSP.Type.CG)
ksp.setNormType(psc.KSP.NormType.NORM_NATURAL)
ksp.setTolerances(rtol=1e-6, atol=0, divtol=1e16, max_it=400)
ksp.solve(v2,v1)   

print ("petsc-its =", ksp.its)
print ("petsc-dot = ", v1.dot(v2))